In [34]:
import collections
import pickle

import os
import numpy as np
import pandas as pd

import tensorflow as tf

In [35]:
tf.config.experimental.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [36]:
def savepickle(fname,*args):
    with open(fname+"_pk","wb") as f:
        pickle.dump(args,f)
        
def loadpickle(fname):
    with open(fname,"rb") as f:
        obj = pickle.load(f)
        
    return obj

def reformat(x):
    """
    input: x = array([[array([5.]), 1])
    output: x = array([5,0,0,0,0])
    """
    x = list(x[0])
    #只截取5个，因为模型只输出5个
    x = x[:5]
    p = len(x)
    x = x + [0]*(5-p)
    return np.array(x)

def load_data(rootdir,pk="digitStruct.mat_pk",num_only=True):
    #pk_path = os.path.join(rootdir,pk)
    image_names,labels = loadpickle(pk)
    
    labels_x_len = labels[:,-1:] 
    labels_x_len[labels_x_len>5]=6
    labels_x_len = labels_x_len.astype(float)
    labels_num = np.apply_along_axis(reformat,1,labels)
    labels = np.concatenate((labels_x_len,labels_num),axis=1)
    image_names = np.array([os.path.join(rootdir,x) for x in image_names])
    if num_only:
        return image_names,labels_num

    return image_names,labels

def to_df(x,y):
    xdf = pd.DataFrame(x,columns=["filename"])
    #这样有问题，应该 dataset = pd.DataFrame({'Column1': data[:, 0], 'Column2': data[:, 1]})
    ydf = pd.DataFrame(y,columns=["len","1","2","3","4","5"])
    ydf = ydf.astype(int)
    
    return pd.concat([xdf,ydf],axis=1)

def to_one_hot(n_arr,cls_num):
    onehot = np.zeros((n_arr.shape[1],cls_num))
    onehot[np.arange(n_arr.shape[1]),n_arr]=1
    return onehot

 
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [32, 32])
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def datset_gen(images,lables,batch_size=16,buffer_size=1000):

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    path_ds = tf.data.Dataset.from_tensor_slices(images)
    image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
    label_ds = tf.data.Dataset.from_tensor_slices(lables)
    img_lab_ds = tf.data.Dataset.zip((image_ds, label_ds))

    return img_lab_ds.shuffle(buffer_size=buffer_size).repeat().batch(batch_size)

In [37]:
import tensorflow as tf
from tensorflow import keras
from functools import partial
import numpy as np 
from tensorflow.keras.layers import Flatten,Dense,Activation,MaxPool2D,GlobalAvgPool2D,BatchNormalization
from tensorflow.keras.layers import Input,Conv2D,Lambda,Dropout
from tensorflow.keras import Model


DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,kernel_initializer='random_uniform',
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

def ResNet34_only(input):

    """
    Use ResNet34 instead of the orig

    """ 
    #CNN network
    X = DefaultConv2D(64, kernel_size=7, strides=2,kernel_initializer='he_normal')(input)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D(pool_size=3, strides=2, padding="SAME")(X)
    prev_filters = 64
    for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
        strides = 1 if filters == prev_filters else 2
        X = ResidualUnit(filters, strides=strides)(X)
        prev_filters = filters
    X = GlobalAvgPool2D()(X)
    y = Flatten()(X)

    #Outputs
    return y 




def ResNet34(input,N=5,class_num=11):

    """
    Use ResNet34 instead of the orig

    """ 
    #CNN network
    X = DefaultConv2D(64, kernel_size=7, strides=2)(input)
    #X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D(pool_size=3, strides=2, padding="SAME")(X)
    prev_filters = 64
    for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
        strides = 1 if filters == prev_filters else 2
        X = ResidualUnit(filters, strides=strides)(X)
        prev_filters = filters
    X = GlobalAvgPool2D()(X)
    y = Flatten()(X)

    #
    L = Dense(N+2,activation="softmax")(y)
    S = [ Dense(class_num,activation="softmax")(y) for _ in range(N) ]

    #Outputs
    return Model(inputs=[input],outputs=[L]+S)

######################
######################

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

@tf.function
def loss(model, x, y):
    y_hat = model(x)
    #iterating over `tf.Tensor` is not allowed: AutoGraph did not convert this function.
    #loss = [loss_object(t,p) for t,p in zip(y,y_hat)]
    #loss = loss[0] + tf.reduce_mean(loss[1:])
    p_loss = tf.reduce_mean(loss_object(y[:,0],y_hat[0]))
    loss = [p_loss]
    for i in range(1,6):
        loss.append( tf.reduce_mean(loss_object(y[:,i],y_hat[i])) )
      
    return loss

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

def svhn_train(input_ds,
               num_epochs=1,
               learning_rate=0.001,
               input_shape=[32,32,3],
               CNNModel=ResNet34,
               N=5,class_num=11):

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,clipvalue=1)
    AUTOTUNE = tf.data.experimental.AUTOTUNE

    train_loss_results = []
    train_accuracy_results = []
    

    train_log_string="Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}"

    X = Input(shape=input_shape)
    model = CNNModel(X,N,class_num)
    print("==========CNNModel Created!!!============")
    for epoch in range(num_epochs):
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

        for x,y in input_ds.prefetch(buffer_size=AUTOTUNE).take(1000):
            
            # 优化模型
            loss_value, grads = grad(model, x, y)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # 追踪进度
            epoch_loss_avg(loss_value)  # 添加当前的 batch loss
            # 比较预测标签与真实标签
            #epoch_accuracy(y, model(x))
            print(train_log_string.format(epoch,epoch_loss_avg.result(),
                                              epoch_accuracy.result()))
            
      
        # 循环结束
        train_loss_results.append(epoch_loss_avg.result())
        train_accuracy_results.append(epoch_accuracy.result())

        print(train_log_string.format(epoch,epoch_loss_avg.result(),
                                              epoch_accuracy.result()))
      
        if epoch % 50 == 0:
            print(train_log_string.format(epoch,epoch_loss_avg.result(),
                                              epoch_accuracy.result()))
        #ToDo:
        #1.valid during training
        #2.callback:save(),log
        #for fn in callback:
        #    pass
    
    return model

In [38]:
def CNN_org(input):
    X = DefaultConv2D(64, kernel_size=4, strides=2)(input)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = MaxPool2D(pool_size=3, strides=2, padding="SAME")(X)
    prev_filters = 64
    for filters in [64] * 1 + [128] * 2 + [256] * 2 + [512] * 1:
        strides = 1 if filters == prev_filters else 2
        X = DefaultConv2D(filters, kernel_size=2, strides=strides)(X)
        #X = BatchNormalization()(X)
        X = Activation("relu")(X)
        X = MaxPool2D(pool_size=2, strides=1, padding="SAME")(X)
        X = Dropout(0.1)(X)
    y = Flatten()(X)

    #Outputs
    return y

def svhn_model_simple(input_shape=[128,128,3],N=5,class_num=11):
    X=Input(shape=input_shape)
    y = CNN_org(X)
    y = Dense(20,activation="relu",kernel_initializer='random_uniform')(y)
    S = [ Dense(class_num,activation="softmax",kernel_initializer='random_uniform')(y) for _ in range(N) ]
    S = tf.stack(S,axis=1)
    #S = Dense(N+2,activation="softmax",kernel_initializer='he_normal')(y)
    return Model(inputs=X,outputs=S)

## Train

In [39]:
root_path = "/kaggle/input/street-view-house-numbers/"
train_path ="/kaggle/input/svhnpk/train_digitStruct.mat_pk"
test_path =  "/kaggle/input/svhnpk/test_digitStruct.mat_pk"
extra_path = "/kaggle/input/svhnpk/extra_digitStruct.mat_pk"
is_num_only=True
X_test,y_test = load_data(root_path+"test/test/",test_path,num_only=is_num_only)
X_train,y_train = load_data(root_path+"train/train/",train_path,num_only=is_num_only)
X_extra,y_extra = load_data(root_path+"extra/extra/",extra_path,num_only=is_num_only)

X_train = np.concatenate([X_train,X_extra])
y_train = np.concatenate([y_train,y_extra])
y_train = y_train.reshape((-1,5,1))
y_test = y_test.reshape((-1,5,1))

In [40]:
batch_size  = 64
ds_train = datset_gen(X_train,y_train,batch_size=batch_size,buffer_size=100)
ds_test = datset_gen(X_test,y_test,batch_size=batch_size,buffer_size=100)

In [41]:
# model = svhn_train(ds_train,
#                num_epochs=100,
#                learning_rate=0.01,
#                input_shape=[32,32,3],
#                CNNModel=ResNet34,
#                N=5,class_num=11)

In [42]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,8))
# for n, image in enumerate(ds_train.take(1)):
#   plt.subplot(2,2,n+1)
#   plt.imshow(image)
#   plt.grid(False)
#   plt.xticks([])
#   plt.yticks([])
#   plt.xlabel(caption_image(all_image_paths[n]))
#   plt.show()

In [43]:
loss_cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)



model = svhn_model_simple(input_shape=[32,32,3])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1,clipvalue=1)

# - 该模型5个位置，分别计算LOSS
model.compile(optimizer=optimizer,
              loss=loss_cce,
              metrics=['accuracy'])

In [44]:

#tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
#checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath="checkpoint_"+model_save_file, verbose=1)

In [45]:
#model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 16, 16, 64)   3072        input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_38 (BatchNo (None, 16, 16, 64)   256         conv2d_52[0][0]                  
__________________________________________________________________________________________________
activation_17 (Activation)      (None, 16, 16, 64)   0           batch_normalization_38[0][0]     
____________________________________________________________________________________________

In [ ]:
num_epoches=100
model_save_file="temp.h5"
with tf.device("/GPU:0"):
    history = model.fit_generator(ds_train,
                steps_per_epoch=X_train.shape[0]//batch_size,
                epochs=num_epoches,
                validation_data=ds_test,
                validation_steps=100)

Train for 3683 steps, validate for 100 steps
Epoch 1/100
3683/3683 [==============================] - 264s 72ms/step - loss: 2.0530 - accuracy: 0.5549 - val_loss: 1.8924 - val_accuracy: 0.6403
Epoch 2/100
 269/3683 [=>............................] - ETA: 2:29 - loss: 1.9278 - accuracy: 0.6063

In [ ]:
weights = model.get_weights()

In [48]:
weights

[array([[[[-0.02804869, -0.01341884, -0.01940371, ...,  0.0364545 ,
            0.01134186,  0.03503695],
          [ 0.00362255,  0.04303589, -0.04238228, ..., -0.03015273,
            0.04890195,  0.03636086],
          [ 0.0057392 , -0.00814781, -0.03299191, ...,  0.01787943,
            0.0238248 , -0.02398976]],
 
         [[-0.00381555,  0.04930308,  0.03755447, ...,  0.02552018,
           -0.00625777,  0.0113618 ],
          [ 0.00357817, -0.01672465,  0.04494112, ..., -0.00498785,
            0.04193885,  0.00399535],
          [ 0.00828748, -0.02260344, -0.02970089, ..., -0.04704617,
            0.04525036, -0.01397057]],
 
         [[ 0.00404362,  0.0397182 ,  0.02969008, ...,  0.03405868,
            0.03420844, -0.03210963],
          [-0.04965442,  0.02611503, -0.03455722, ...,  0.00207044,
            0.01359465, -0.04760742],
          [-0.04267329,  0.01785368, -0.03910087, ...,  0.03726543,
           -0.04810413,  0.03819713]],
 
         [[-0.04771626,  0.04176437, 

In [49]:
model.save("model.cnnorg.h5")
savepickle("weitgh_only.h5",weights)

#-- visualize --
import matplotlib.pyplot as plt

#import matplotlib as mpl
#mpl.rcParams['figure.figsize'] = (8, 6)
#mpl.rcParams['axes.grid'] = False

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(num_epoches)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

NameError: name 'history' is not defined